In [ ]:
import os
import cv2
import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from geopandas import GeoSeries
from shapely.geometry import Polygon
from rasterio.windows import Window
from rasterio.plot import reshape_as_image


%matplotlib inline

2 most usefull types of satellite imagery:

* Sentinel-2
    
    Max - 10 meters per pixel
    
    Download tiles: https://scihub.copernicus.eu/dhus/#/home




* Landsat-8

    Max - 30 meters per pixel
    
    Download tiles: https://earthexplorer.usgs.gov/

QGIS. Check the data

Reading Raster with rasterio

In [ ]:
PROJECT_DIR = '/home/ymi/data/ucu_data'

RASTER_PATH = os.path.join(PROJECT_DIR, 'T34JEP_20170101T082332/T34JEP_20170101T082332_TCI.jp2')
TRAIN_POLYGONS_PATH = os.path.join(PROJECT_DIR, 'train-20220726T194123Z-001/train/train.shp')
TEST_POLYGONS_PATH = os.path.join(PROJECT_DIR, 'train-20220726T194123Z-001/test/test.shp')
RASTER_MASK_PATH = os.path.join(PROJECT_DIR, 'mask.jp2')
FRAGMENT_STORAGE = os.path.join(PROJECT_DIR, 'split')

In [ ]:
#read image
with rasterio.open(RASTER_PATH, "r") as src:
    raster = src.read()
    metadata = src.meta

In [ ]:
src.crs

In [ ]:
#check meta
print(metadata)

In [ ]:
#plot image
raster_image = reshape_as_image(raster)
plt.imshow(raster_image)

In [ ]:
#read train 
train_df = gpd.read_file(TRAIN_POLYGONS_PATH)
test_df = gpd.read_file(TEST_POLYGONS_PATH)

In [ ]:
train_df = pd.concat([train_df, test_df])
train_df.reset_index(inplace=True, drop=True)

In [ ]:
#visualize polygon
train_df['geometry'][0]

In [ ]:
train_df['geometry'][0].exterior.coords.xy

In [ ]:
train_df['Field_Id'].isin(list(range(300, 401))).sum()

In [ ]:
src.bounds

In [ ]:
src = rasterio.open(RASTER_PATH, 'r')
failed = []
for num, row in train_df.iterrows():
    try:
        # mask raster
        masked_image, out_transform = rasterio.mask.mask(src, [mapping(row['geometry'])], crop=True, nodata=0)
    except Exception as error:
        print(error)
        failed.append(num)
print("Rasterio failed to mask {} files".format(len(failed)))

1. Go to http://projfinder.com/
2. We know the data came from South Africa let’s zoom into it.
3. Use it with any coordinates (for example, X: 2467881.175041331 Y: -3352032.059296422) from the GeoDataframe and check the output. We are looking for a place in South Africa, that has a river — we can see it in our image. Looking through the results we will see one that fits: EPSG:3395 Name: WGS 84 / World Mercator.

In [ ]:
# assigning crs
train_df.crs = {'init' :'epsg:3395'}


In [ ]:
train_df.crs

In [ ]:
metadata.get('crs').to_epsg()

In [ ]:
# let's remove rows without geometry
train_df = train_df[train_df.geometry.notnull()]

# assigning crs
train_df.crs = {'init' :'epsg:3395'}

#transforming polygons to the raster crs
train_df = train_df.to_crs(metadata.get('crs').to_epsg())

In [ ]:
train_df.crs

In [ ]:
train_df['geometry'][0].exterior.coords.xy

In [ ]:
TRAIN_POLYGONS_CONVERTED = os.path.join(PROJECT_DIR, 'train-20220726T194123Z-001/train/train.geojson')

train_df.to_file(TRAIN_POLYGONS_CONVERTED, driver="GeoJSON")